In [10]:
import os, glob
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import masking
import matplotlib.pyplot as plt
import scipy.stats
import neural_helpers as nh

%load_ext autoreload
%autoreload 2

nibs_base = '/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries'

# Extract & normalize hippocampal patterns
(smooth? ~2mm kernel)

In [27]:
test = np.load('/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid01/func/sub-hybrid01_task-main_run-1_space-MNI152NLin2009cAsym_desc-choice_betaseries_hipp.npy')
test.mean(axis=1).shape

(400,)

In [ ]:
base = '/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries'

import glob
hipp_files = glob.glob(base + '/sub-hybrid??/func/*.npy')
for fi 
from joblib import Parallel, delayed

def norm_one_pattern(file): # timepoints x voxels
    pattern = np.load(file)
    normalized = (pattern - np.mean(pattern, axis=1, keepdims=True)) / np.std(pattern, axis=1, keepdims=True)
    outpath = file.replace('hipp.npy', 'hipp_norm.npy')
    np.save(outpath, normalized)
    os.remove(file)

Parallel(n_jobs=4)( delayed(norm_one_pattern)(f) for f in hipp_files )

/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-fb_betaseries_hipp_norm.npy
/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-choice_betaseries_hipp_norm.npy


In [ ]:
paths = ['/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-fb_betaseries_hipp_norm.npy',
         '/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-choice_betaseries_hipp_norm.npy'
]
beta_paths = ['/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-fb_betaseries_betaseries.nii.gz',
              '/Volumes/shohamy-locker/chris/hybrid_mri_bids/derivatives/nibetaseries/sub-hybrid13/func/sub-hybrid13_task-main_run-3_space-MNI152NLin2009cAsym_desc-choice_betaseries_betaseries.nii.gz'
]
path = paths[0]

beta_data = nib.load(beta_file).get_fdata()

# Pattern analyses

In [ ]:
beh = pd.read_csv('./data/hybrid_data.csv')
# remember to exclude trials from first 6 volumes that don't exist in fmri data

1) Correlation of encoding to retrieval trials - compare optimal to non-optimal memory

In [ ]:
# encoding choice trial

# encoding feedback trial

# does ERS increase with time?

2) Similarity kernel of retrieval trial to those around encoding trial. Does this similarity predict old/new trial performance? old vs. new choices, optimal vs. nonoptimal

- Predicting value
- lucky vs. unlucky deck (value matched) on either encoding or retrieval
- save vs. different luck state encoding vs. retrieval
- predict subsequent retrieval from pattern at feedback of choice image